# Network dimensionality and universality of rules

## For every considered model, vary (1) number and (2) width of layers. Otherwise same as 'comparing'.

Created by Basile Van Hoorick, Fall 2020.

In [1]:
%run FF_common.ipynb

In [2]:
def get_metrics(hyperparams, all_stats):

    num_configs = len(hyperparams)
    test_accs_up = []
    test_accs_down = []
    speeds_down = []

    for i in range(num_configs):

        universal, l, w, cap = hyperparams[i]
    #     print('Universal:', universal)
    #     print('Number hidden layers:', l)
    #     print('Hidden layer width:', w)
    #     print('Cap:', cap)

        stats_up, stats_down = all_stats[i]
        stats_up = convert_multi_stats_uncertainty(stats_up)
        stats_down = convert_multi_stats_uncertainty(stats_down)
        test_acc_up = stats_up[2][-1]
        test_acc_down = stats_down[2][-1]
        speed_down = np.mean(stats_down[2]) / test_acc_down

    #     print(f'Final upstream test accuracy: {test_acc_up*100:.2f}%')
    #     print(f'Final downstream test accuracy: {test_acc_down*100:.2f}%')
    #     print(f'Downstream speed of convergence: {speed_down*100:.2f}%')
    #     print()

        test_accs_up.append(test_acc_up)
        test_accs_down.append(test_acc_down)
        speeds_down.append(speed_down)

    test_accs_up = np.array(test_accs_up).reshape((2, 3, 3))
    test_accs_down = np.array(test_accs_down).reshape((2, 3, 3))
    speeds_down = np.array(speeds_down).reshape((2, 3, 3))
    metrics = np.multiply(test_accs_up, test_accs_down, speeds_down)

    print('Upstream accuracy:')
    print(test_accs_up)
    print('Downstream accuracy:')
    print(test_accs_down)
    print('Downstream speed:')
    print(speeds_down)
    print('Product:')
    print(metrics)

    return metrics

In [3]:
# IMPORTANT: Henceforth, we use GD directly on inputs but use plasticity rules in the output and hidden layers.
opts = Options(gd_input=True,
               use_graph_rule=True,
               gd_graph_rule=True,
               use_output_rule=True,
               gd_output_rule=True,
               gd_output=False)
scheme = UpdateScheme(cross_entropy_loss=True,
                      mse_loss=False,
                      update_misclassified_only=False,
                      update_all_edges=True)

# Feed-forward brain config.
n_up = 16  # Input layer size for meta-learning.
n_down = 16  # Input layer size for desired task training.
n = 16  # Input layer size.
m = 2  # Output layer size.
p = 0.5  # Connectivity probability.

# Training config.
num_runs = 5
num_rule_epochs = 50
num_epochs_upstream = 1
num_epochs_downstream = 1
dataset = 'halfspace'

## PrePost

In [ ]:
hyperparams = []
all_stats = []

for universal in [False, True]:  # Specialized rules or not?
    
    for l in [2, 3, 4]:  # Number of hidden layers.
        
        for w in [16, 32, 64]:  # Width of hidden layers.
            
            cap = w // 2  # Number of nodes firing per layer.
            print('Universal:', universal)
            print('Number hidden layers:', l)
            print('Hidden layer width:', w)
            print('Cap:', cap)
            hyperparams.append((universal, l, w, cap))
            
            brain_prepost_fact = lambda: FFLocalNet(
                n, m, l, w, p, cap,
                hl_rules=TableRule_PrePost() if universal else [TableRule_PrePost()] * (l-1),
                output_rule=TableRule_PrePost(), options=opts, update_scheme=scheme)
            
            print('==== Interpretation: Pre and Post ====')
            cur_stats = evaluate_brain(
                brain_prepost_fact, n, dataset=dataset,
                num_runs=num_runs, num_rule_epochs=num_rule_epochs,
                num_epochs_upstream=num_epochs_upstream, num_epochs_downstream=num_epochs_downstream)
            all_stats.append(cur_stats)

  0%|          | 0/50 [00:00<?, ?it/s]

Universal: False
Number hidden layers: 2
Hidden layer width: 16
Cap: 8
==== Interpretation: Pre and Post ====

Run 1 / 5...
Meta-learning...


  4%|▎         | 54/1500 [00:00<00:02, 496.92it/s]

Last loss: 0.3789
Last train accuracy: 0.9347
Last test accuracy: 0.9120
Training SAME brain instance (NO backprop)...
===> WARNING: Backprop is disabled, which means that all layers without rules will never change their weights!
===> This is NOT recommended by Basile!
INITIAL train accuracy: 0.4940
INITIAL test accuracy: 0.4600
Epoch 1 / 1 ...


  0%|          | 0/50 [00:00<?, ?it/s]

...

  3%|▎         | 46/1500 [00:00<00:03, 459.76it/s]

Last loss: 0.3597
Last train accuracy: 0.9653
Last test accuracy: 0.9380
Training SAME brain instance (NO backprop)...
===> WARNING: Backprop is disabled, which means that all layers without rules will never change their weights!
===> This is NOT recommended by Basile!
INITIAL train accuracy: 0.4980
INITIAL test accuracy: 0.4720
Epoch 1 / 1 ...


  0%|          | 0/50 [00:00<?, ?it/s]


Last loss: 0.6788
Last train accuracy: 0.6500
Last test accuracy: 0.6640

Universal: False
Number hidden layers: 2
Hidden layer width: 64
Cap: 32
==== Interpretation: Pre and Post ====

Run 1 / 5...
Meta-learning...


 66%|██████▌   | 33/50 [01:29<00:44,  2.61s/it]

In [ ]:
metrics_prepost = get_metrics(hyperparams_prepost, all_stats_prepost)

Three best configs:

* Universal, 2 layers, 64 width (metric 0.69)

* Non-universal, 4 layers, 32 width (metric 0.66)

* Universal, 2 layers, 32 width (metric 0.62)

## PrePostCount

In [ ]:
hyperparams_prepostcount = []
all_stats_prepostcount = []

for universal in [False, True]:  # Specialized rules or not?
    
    for l in [2, 3, 4]:  # Number of hidden layers.
        
        for w in [16, 32, 64]:  # Width of hidden layers.
            
            cap = w // 2  # Number of nodes firing per layer.
            print('Universal:', universal)
            print('Number hidden layers:', l)
            print('Hidden layer width:', w)
            print('Cap:', cap)
            hyperparams_prepostcount.append((universal, l, w, cap))
            
            brain_prepostcount_fact = lambda: FFLocalNet(
                n, m, l, w, p, cap,
                hl_rules=TableRule_PrePostCount() if universal else [TableRule_PrePostCount()] * (l-1),
                output_rule=TableRule_PrePostCount(), options=opts, update_scheme=scheme)
            
            print('==== Interpretation: PrePostCount ====')
            cur_stats = evaluate_brain(
                brain_prepostcount_fact, n, dataset=dataset,
                num_runs=num_runs, num_rule_epochs=num_rule_epochs,
                num_epochs_upstream=num_epochs_upstream, num_epochs_downstream=num_epochs_downstream)
            all_stats_prepostcount.append(cur_stats)

In [ ]:
metrics_prepostcount = get_metrics(hyperparams_prepostcount, all_stats_prepostcount)

## PrePostPercent

In [ ]:
hyperparams_prepostpercent = []
all_stats_prepostpercent = []

for universal in [False, True]:  # Specialized rules or not?
    
    for l in [2, 3, 4]:  # Number of hidden layers.
        
        for w in [16, 32, 64]:  # Width of hidden layers.
            
            cap = w // 2  # Number of nodes firing per layer.
            print('Universal:', universal)
            print('Number hidden layers:', l)
            print('Hidden layer width:', w)
            print('Cap:', cap)
            hyperparams_prepostpercent.append((universal, l, w, cap))
            
            brain_prepostpercent_fact = lambda: FFLocalNet(
                n, m, l, w, p, cap,
                hl_rules=TableRule_PrePostPercent() if universal else [TableRule_PrePostPercent()] * (l-1),
                output_rule=TableRule_PrePostPercent(), options=opts, update_scheme=scheme)
            
            print('==== Interpretation: PrePostPercent ====')
            cur_stats = evaluate_brain(
                brain_prepostpercent_fact, n, dataset=dataset,
                num_runs=num_runs, num_rule_epochs=num_rule_epochs,
                num_epochs_upstream=num_epochs_upstream, num_epochs_downstream=num_epochs_downstream)
            all_stats_prepostpercent.append(cur_stats)

In [ ]:
metrics_prepostpercent = get_metrics(hyperparams_prepostpercent, all_stats_prepostpercent)